Using this  - [Notebook](https://github.com/Arize-ai/phoenix/blob/main/tutorials/llm_ops_overview.ipynb) - as a jumping off point

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:

import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:

import phoenix as px
from llama_index.core import set_global_handler

# Setup phoenix tracing
px.launch_app()
set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
import os
from getpass import getpass

import phoenix as px
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage,
    VectorStoreIndex
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser


Settings.llm = OpenAI(model="gpt-3.5-turbo-0125")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [5]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine()

In [7]:

from tqdm import tqdm

queries = [
    "What is Paul Graham's contribution to computer science?",
    "What startups has Paul Graham founded?",
    "What is the impact of Paul Graham's Y Combinator on the tech industry?",
    "What are some notable essays written by Paul Graham?"
]

for query in tqdm(queries):
    response = query_engine.query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")

 25%|██▌       | 1/4 [00:01<00:05,  2.00s/it]

Query: What is Paul Graham's contribution to computer science?
Response: Paul Graham's contribution to computer science is his focus on Lisp and his book "On Lisp." He emphasized the importance of Lisp for its own sake, beyond just its association with AI, and decided to write a book about Lisp hacking. This book, "On Lisp," which he worked on during grad school and was published in 1993, showcases his efforts to delve into Lisp and share his insights with others in the field.


 50%|█████     | 2/4 [00:03<00:02,  1.50s/it]

Query: What startups has Paul Graham founded?
Response: Paul Graham founded Y Combinator, which is a startup accelerator that has funded numerous successful startups such as Reddit, Twitch, and others mentioned in the provided context.


 75%|███████▌  | 3/4 [00:05<00:01,  1.75s/it]

Query: What is the impact of Paul Graham's Y Combinator on the tech industry?
Response: Paul Graham's Y Combinator had a significant impact on the tech industry by introducing the batch model of funding startups twice a year and providing intensive support to these startups for three months. This approach helped make starting a startup more accessible and common, challenging the traditional customs of venture capital that were still rooted in the past. By focusing on helping founders and creating a supportive environment for startups, Y Combinator contributed to fostering innovation and entrepreneurship in the tech industry.


100%|██████████| 4/4 [00:06<00:00,  1.61s/it]

Query: What are some notable essays written by Paul Graham?
Response: Some notable essays written by Paul Graham include those discussing the use of Lisp at Viaweb, reflections on the changing landscape of publishing essays online, and insights on the value of working on unprestigious endeavors.
